# 20240805 - Liver transplant analysis

This notebook has the code necessary to reproduce the findings in the South Dakota News Watch article, ["TK,"](TK) published Aug. 5, 2024.

This article includes findings from analyzing portions of the [UNOS STAR file](https://optn.transplant.hrsa.gov/data/view-data-reports/request-data/). See [README.md](README.md) for more details on the data.

- [Setup and summary stats](#Setup-and-summary-stats)
    - [Read in main liver data file](#Read-in-main-liver-data-file)
    - [Read in liver file code lookup](#Read-in-liver-file-code-lookup)
    - [Read in STAR file data dictionary](#Read-in-STAR-file-data-dictionary)
    - [Break out age groups](#Break-out-age-groups)
    - [Break out alcohol diagnoses](#Break-out-alcohol-diagnoses)
    - [Break out South Dakota data](#Break-out-South-Dakota-data)
- [Findings](#Findings)
  - ["Almost two of every three liver donor candidates ultimately received a transplant."](#Almost-two-of-every-three-liver-donor-candidates-ultimately-received-a-transplant.)
  - ["Most liver transplant patients aren't this young. ... Two thirds \[of candidates in their 30s\] received a transplant."](#Most-liver-transplant-patients-aren't-this-young.)
  - [Sentence TK about 30-year-olds with alcohol diagnoses](#Sentence-TK-about-30-year-olds-with-alcohol-diagnoses)
  - [Sentence TK about the changing rate of alcohol diagnoses](#Sentence-TK-about-the-changing-rate-of-alcohol-diagnoses)

## Setup and summary stats

In [1]:
import math
from datetime import date

from utils import get_df

import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

#### Read in main liver data file

In [3]:
# load the LIVER_DATA.DAT file
df = get_df(
    'data/liver/LIVER_DATA.DAT',
    parse_dates=[
        'PX_STAT_DATE',
        'INIT_DATE'
    ]
)

In [4]:
# how many records are missing a list year?
len(df[df['LISTYR'].isnull()])

1167

In [5]:
# We're only going to use data on or after the first full year of listing data: 1988
df = df[df['LISTYR'] > 1987]

In [6]:
# how many records total?
total_records = len(df)
print(f'{total_records:,} events')

367,012 events


In [7]:
# any nulls in patient ID column?
print(len(df[df['PT_CODE'].isnull()]), 'nulls')

0 nulls


In [8]:
# what are max and min dates (formatted)?
date_first = df['INIT_DATE'].min().strftime("%B %Y")
date_last = df['INIT_DATE'].max().strftime("%B %Y")

# there are a few records past June 30, 2024, but we're going to
# stick with the documentation and cap the data at the end of 6/24
date_last = 'June 2024'

print(f'Earliest event: {date_first}')
print(f'Most recent event: {date_last}')

Earliest event: January 1988
Most recent event: June 2024


In [9]:
# how many unique candidates?
# get the count of unique patient codes
total_candidates_unique_count = df['PT_CODE'].nunique()
print(f'{total_candidates_unique_count:,} candidates')

318,437 candidates


In [10]:
# how many unique donors?
# get the count of unique donor IDs
total_donors_unique_count = df['DONOR_ID'].nunique()
print(f'{total_donors_unique_count:,} donors')

213,641 donors


In [11]:
# how many candidates got a transplant? for this,
# we want to make sure that every record associated with a patient
# shows whether they have (at some point) gotten a transplant

# step one: get a unique list of all PT_CODE values that are
# associated with a transplant record -- i.e., the TRR_ID_CODE
# value is not null
pt_codes_w_tx = df[df['TRR_ID_CODE'].notnull()]['PT_CODE'].unique()

# set a convenience flag on _every_ event record showing whether this
# candidate has ever been associated with a tx record
df['has_tx'] = df['PT_CODE'].isin(pt_codes_w_tx)

### Almost two of every three liver donor candidates ultimately received a transplant.

For all liver donor candidates in this time period, the transplant rate is 63.43%.

In [12]:
df_cand_w_tx = df[df['has_tx']]
total_candidates_w_tx = df_cand_w_tx['PT_CODE'].nunique()

# and get the % of total
tx_pct = total_candidates_w_tx / total_candidates_unique_count

print(f'In the U.S., from {date_first} to {date_last}, {total_candidates_w_tx:,} of {total_candidates_unique_count:,} liver transplant candidates received a transplant ({tx_pct:.2%}).')

In the U.S., from January 1988 to June 2024, 201,973 of 318,437 liver transplant candidates received a transplant (63.43%).


In [13]:
# checks out
df[df['has_tx']]['PT_CODE'].nunique() == df[df['TRR_ID_CODE'].notnull()]['PT_CODE'].nunique()

True

#### Read in liver file code lookup

We can use this to look up pieces of encoded data.

In [14]:
df_code_dict = get_df(
    'docs/CODE DICTIONARY - FORMATS 202406/Liver/LIVER_FORMATS_FLATFILE.DAT',
    encoding='cp1252'
)

In [15]:
df_code_dict.head()

,LABEL,FMTNAME,TYPE,CODE
0,O,ABO,C,1
1,A,ABO,C,2
2,B,ABO,C,3
3,AB,ABO,C,4
4,A1,ABO,C,5


#### Read in STAR file data dictionary

Mainly to check the variable start/end dates.

In [16]:
df_star_data_dict = pd.read_excel(
    'docs/STAR User Guide/STAR File Data Dictionary.xlsx',
    sheet_name='LIVER_DATA',
    skiprows=1
)

In [17]:
df_star_data_dict.head()

,VARIABLE NAME,DESCRIPTION,FORM,VAR START DATE,VAR END DATE,FORM SECTION,DATA TYPE,SAS ANALYSIS FORMAT,COMMENT
0,ABO,RECIPIENT BLOOD GROUP @ REGISTRATION,TCR,1987-10-01,NaT,CLINICAL INFORMATION,CHAR(3),ABO,NaN
1,ABO_DON,DONOR BLOOD TYPE,DDR/LDR,1987-10-01,NaT,DONOR INFORMATION,CHAR(3),ABO,NaN
2,ABO_MAT,DONOR-RECIPIENT ABO MATCH LEVEL,CALCULATED,NaT,NaT,NaN,CHAR(1),ABOMAT,NaN
3,ACADEMIC_LEVEL_TCR,ACADEMIC ACTIVITY LEVEL AT LISTING,TCR,2004-06-30,NaT,CANDIDATE INFORMATION,NUM,ACADLVLKI,NaN
4,ACADEMIC_LEVEL_TRR,ACADEMIC ACTIVITY LEVEL AT TRANSPLANT,TRR,2004-06-30,NaT,PATIENT STATUS,NUM,ACADLVLKI,NaN


#### Break out age groups

Part of our analysis involves looking at candidates in age buckets: twenties, thirties, etc.

In [18]:
# we want to bin by decades -- 10-year intervals
interval = 10

# get the highest candidate age, rounded up to the nearest 10
age_max = math.ceil(df['INIT_AGE'].max() / float(interval)) * interval

# get age groups as a range
age_bins = list(range(0, age_max + interval, interval))

# and add labels
labels = [f'{x}-{x + interval - 1}' for x in age_bins[:-1]]

print(age_bins)
print(labels)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']


In [19]:
df['candidate_age_decade'] = pd.cut(
    df['INIT_AGE'],
    bins=age_bins,
    labels=labels
)

In [20]:
df['candidate_age_decade'].value_counts()

candidate_age_decade
50-59    123855
60-69     84147
40-49     78088
30-39     29824
20-29     12724
0-9       11999
10-19      9576
70-79      6799
80-89        12
Name: count, dtype: int64

#### Break out alcohol diagnoses

Candidates who were listed with a primary or secondary diagnosis associated with an alcohol-related disease will have one of these codes present in the `DGN_TCR`, `DGN2_TCR` or `DIAG` fields, according to UNOS:
- 4215 – Alcoholic Cirrhosis
- 4216 – Alcoholic Cirrhosis with Hepatitis C
- 4217 – Acute Alcoholic Hepatitis
- 4218 – Acute Alcohol-Associated Hepatitis with or without Cirrhosis
- 4219 – Alcohol-Associated Cirrhosis without Acute Alcohol-Associated Hepatitis

Here, we're going to check for the presence of these codes and set a flag on each record.

In [21]:
# check the variable start dates for those columns
df_star_data_dict[df_star_data_dict['VARIABLE NAME'].isin(['DGN_TCR', 'DGN2_TCR', 'DIAG'])]

,VARIABLE NAME,DESCRIPTION,FORM,VAR START DATE,VAR END DATE,FORM SECTION,DATA TYPE,SAS ANALYSIS FORMAT,COMMENT
87,DGN_TCR,PRIMARY DIAGNOSIS AT TIME OF LISTING,TCR,1994-04-01,NaT,CLINICAL INFORMATION,NUM,ALL_DGN,NaN
88,DGN2_TCR,SECONDARY DIAGNOSIS AT TIME OF LISTING,TCR,1994-04-01,NaT,CLINICAL INFORMATION,NUM,ALL_DGN,NaN
92,DIAG,RECIPIENT PRIMARY DIAGNOSIS,TRR>TCR,1987-10-01,NaT,PATIENT STATUS/CLINICAL INFORMATION,NUM,ALL_DGN,Primary diagnosis was not collected on TCR unt...


In [25]:
# check - see which code labels include the word 'alcohol'
df_code_dict[df_code_dict['LABEL'].fillna('').str.contains('alcohol', case=False)]

,LABEL,FMTNAME,TYPE,CODE
2582,ALCOHOLIC CIRRHOSIS,LI_DGN,N,4215
2583,ALCOHOLIC CIRRHOSIS WITH HEPATITIS C,LI_DGN,N,4216
2584,ACUTE ALCOHOLIC HEPATITIS,LI_DGN,N,4217
2585,ACUTE ALCOHOL-ASSOCIATED HEPATITIS WITH OR WIT...,LI_DGN,N,4218
2586,ALCOHOL-ASSOCIATED CIRRHOSIS WITHOUT ACUTE ALC...,LI_DGN,N,4219


In [26]:
alcohol_codes = set([int(x) for x in df_code_dict[df_code_dict['LABEL'].fillna('').str.contains('alcohol', case=False)]['CODE']])

In [27]:
alcohol_codes

{4215, 4216, 4217, 4218, 4219}

In [28]:
def has_alcohol_diagnosis(row):
    ''' function to check a row of liver data to see if any of the alcohol codes are present '''
    diag_codes = [
        row['DGN_TCR'],
        row['DGN2_TCR'],
        row['DIAG']
    ]

    if set(diag_codes).intersection(alcohol_codes):
        return True

    return False

In [29]:
df['alcohol_diagnosis'] = df.apply(
    has_alcohol_diagnosis,
    axis=1
)

In [30]:
# to guard against the possibility of a candidate record not reflecting the alcohol diagnostic
# code for that candidate, set the 'alcohol_diagnosis' flag on ALL records for candidates who had an
# alcohol diagnosis in ANY of their records

# (belt and suspenders -- our numbers didn't change after we used this approach)

pt_codes_w_alc = df[df['alcohol_diagnosis']]['PT_CODE'].unique()

# now set a convenience flag on each event record -- has this patient
# ever been associated with an alcohol diagnosis record?
df['alcohol_diagnosis'] = df['PT_CODE'].isin(pt_codes_w_alc)

In [31]:
# how many candidates were listed with an alcohol diagnosis?
df_alcohol_diag = df[df['alcohol_diagnosis']]
alcohol_cand_total = df_alcohol_diag['PT_CODE'].nunique()

In [32]:
alcohol_cand_pct = alcohol_cand_total / total_candidates_unique_count

print(f'Of {total_candidates_unique_count:,} liver donation candidates across the U.S., {alcohol_cand_total:,} were listed with a primary or secondary diagnosis of an alcohol-related disease ({alcohol_cand_pct:.2%}).')

Of 318,437 liver donation candidates across the U.S., 94,434 were listed with a primary or secondary diagnosis of an alcohol-related disease (29.66%).


In [33]:
df_cands_alcohol_got_tx_count = df_alcohol_diag[df_alcohol_diag['has_tx']]['PT_CODE'].nunique()

In [34]:
df_cands_alcohol_got_tx_pct = df_cands_alcohol_got_tx_count / alcohol_cand_total

print(f'Of those {alcohol_cand_total:,} candidates who were listed with an alcohol diagnosis, {df_cands_alcohol_got_tx_count:,} received a transplant ({df_cands_alcohol_got_tx_pct:.2%}).')

Of those 94,434 candidates who were listed with an alcohol diagnosis, 58,448 received a transplant (61.89%).


In [47]:
# grab a unique list of candidates for use later - get latest record based on
# `PX_START_DATE`
df_uniq_cands = df.sort_values('PX_STAT_DATE', ascending=False).drop_duplicates(
    subset='PT_CODE',
    keep='first'
)

# .. and one for unique donors
df_uniq_donors = df.sort_values('PX_STAT_DATE', ascending=False).drop_duplicates(
    subset='DONOR_ID',
    keep='first'
)

#### Break out South Dakota data

We're using the `PERM_STATE` field to filter for S.D. candidates.

In [38]:
# first, see how many records are missing a candidate home state
missing_cand_state_count = df[df['PERM_STATE'].isnull()]['PT_CODE'].nunique()
missing_cand_state_pct = missing_cand_state_count / total_candidates_unique_count
print(f'{missing_cand_state_count:,} of {total_candidates_unique_count:,} candidates are missing a value for their home state ({missing_cand_state_pct:.2%}).')

4,074 of 318,437 candidates are missing a value for their home state (1.28%).


In [39]:
# filter to get data on S.D. candidates
df_sd = df[df['PERM_STATE'] == 'SD']

# how many events, total?
print(len(df_sd))

776


In [40]:
# how many S.D. candidates, total?
total_candidates_unique_count_sd = df_sd['PT_CODE'].nunique()
print(total_candidates_unique_count_sd)

685


In [41]:
# how many S.D. donors, total?
total_donors_unique_count_sd = df_sd['DONOR_ID'].nunique()
print(total_donors_unique_count_sd)

426


In [42]:
# of the S.D. candidates, who got a liver transplant?
total_candidates_w_tx_sd = df_sd[df_sd['has_tx']]['PT_CODE'].nunique()

# and get the % of total
tx_pct_sd = total_candidates_w_tx_sd / total_candidates_unique_count_sd

print(f'In South Dakota, from {date_first} to {date_last}, {total_candidates_w_tx_sd:,} of {total_candidates_unique_count_sd:,} liver transplant candidates received a transplant ({tx_pct_sd:.2%}).')

In South Dakota, from January 1988 to June 2024, 406 of 685 liver transplant candidates received a transplant (59.27%).


## Findings

### Most liver transplant patients aren't this young.

### Two-thirds [of candidates in their 30s] received a transplant.

Taylor is 35.
- In the U.S. over this time period, 25,148 liver donor candidates were in their 30s when they were listed, of 318,437 total candidates (7.90%). Of those 25,148 candidates in their 30s, 16,557 ultimately got a transplant (65.84%).
- Summary stats for "age at listing" for candidates and "donor age" values across the U.S. in this time period:
  - Candidate age mean: 49.79
  - Candidate age median: 54.0
  - Donor age mean: 37.75
  - Donor age median: 38.0

In [78]:
# how many records are missing `INIT_AGE`?
len(df[df['INIT_AGE'].isnull()])

0

In [79]:
# how many records are missing `DONOR_AGE`? (because not every record is a donation
len(df[df['AGE_DON'].isnull()])

151237

In [80]:
# max/min for `INIT_AGE` (candidate age at listing)
# and `AGE_DON` (donor age)

print('Age range:')
print('- Candidates:', df['INIT_AGE'].min(), '-', df['INIT_AGE'].max())
print('- Donors:', df['AGE_DON'].min(), '-', df['AGE_DON'].max())

Age range:
- Candidates: 0.0 - 86.0
- Donors: 0.0 - 98.0


In [81]:
# grab mean/median age for candidates and donors
age_mean_candidates_us = round(df_uniq_cands['INIT_AGE'].mean(), 2)
age_median_candidates_us = round(df_uniq_cands['INIT_AGE'].median(), 2)

age_mean_donor_us = round(df_uniq_donors['AGE_DON'].mean(), 2)
age_median_donor_us = round(df_uniq_donors['AGE_DON'].median(), 2)

age_str = '\n  - '.join([
    f'Candidate age mean: {age_mean_candidates_us}',
    f'Candidate age median: {age_median_candidates_us}',
    f'Donor age mean: {age_mean_donor_us}',
    f'Donor age median: {age_median_donor_us}'
])

# FINDING
print(f'Nationally:\n  - {age_str}')

Nationally:
  - Candidate age mean: 49.79
  - Candidate age median: 54.0
  - Donor age mean: 37.75
  - Donor age median: 38.0


In [82]:
# just 30-39 YOs, broken down by whether they got a transplant,
df_30s_uniq_cands = df_uniq_cands[df_uniq_cands['candidate_age_decade'] == '30-39']

In [83]:
cand_30s_count = df_30s_uniq_cands['PT_CODE'].nunique()

In [84]:
cand_30s_count

25148

In [70]:
# vet this #
assert(cand_30s_count == len(df_30s_uniq_cands))

In [73]:
# what percentage of all candidates do 30-year-olds represent?
cand_thirties_overall_pct = cand_30s_count / total_candidates_unique_count
print(f'{cand_thirties_overall_pct:.2%}')

7.90%


In [74]:
# of the 30s cohort, how many of them got a transplant?
df_30s_got_tx = df_30s_uniq_cands[df_30s_uniq_cands['has_tx']]

In [75]:
df_30s_got_tx_count = df_30s_got_tx['PT_CODE'].nunique()

In [76]:
df_30s_got_tx_count

16557

In [77]:
cand_30s_tx_pct = df_30s_got_tx_count / cand_30s_count
print(f'In the U.S. over this time period, {cand_30s_count:,} liver donor candidates were in their 30s when they were listed, of {total_candidates_unique_count:,} total candidates ({cand_thirties_overall_pct:.2%}). Of those {cand_30s_count:,} candidates in their 30s, {df_30s_got_tx_count:,} ultimately got a transplant ({cand_30s_tx_pct:.2%}).')

In the U.S. over this time period, 25,148 liver donor candidates were in their 30s when they were listed, of 318,437 total candidates (7.90%). Of those 25,148 candidates in their 30s, 16,557 ultimately got a transplant (65.84%).


### Sentence TK about 30-year-olds with alcohol diagnoses

Most candidates like Taylor, who was in his 30s when he was listed with an alcohol disease-related diagnosis, received a liver transplant.

In [85]:
df_30s_uniq_cands_w_alc = df_30s_uniq_cands[df_30s_uniq_cands['alcohol_diagnosis']]
df_30s_uniq_cands_w_alc_pct = len(df_30s_uniq_cands_w_alc) / cand_30s_count

df_30s_uniq_cands_w_alc_w_tx = df_30s_uniq_cands_w_alc[df_30s_uniq_cands_w_alc['has_tx']]

t30s_uniq_cands_w_alc_w_tx_pct = len(df_30s_uniq_cands_w_alc_w_tx) / len(df_30s_uniq_cands_w_alc)

print(f'Of the {cand_30s_count:,} liver donation candidates who were listed while in their 30s, {len(df_30s_uniq_cands_w_alc):,} had an alcohol disease-related diagnosis ({df_30s_uniq_cands_w_alc_pct:.2%}). Of those {len(df_30s_uniq_cands_w_alc):,} candidates in their 30s with an alcohol diagnosis, {len(df_30s_uniq_cands_w_alc_w_tx):,} got a donated liver ({t30s_uniq_cands_w_alc_w_tx_pct:.2%}). ')


# for 30-year-olds with an alcohol diagnosis, how many and what pct of total candidates actually received a transplant?

Of the 25,148 liver donation candidates who were listed while in their 30s, 9,329 had an alcohol disease-related diagnosis (37.10%). Of those 9,329 candidates in their 30s with an alcohol diagnosis, 6,182 got a donated liver (66.27%). 


### Sentence TK about the changing rate of alcohol diagnoses

The share of liver transplant candidates with a primary or secondary diagnosis of an alcohol-related disease has doubled in the past 30 years.

In [87]:
# note: vetted against manually filtered data as well

# show unique candidates, LISTYR x alcohol diagnosis
df_alc_by_listyr_uniq = pd.pivot_table(
    df_uniq_cands[['LISTYR', 'alcohol_diagnosis']],
    index='alcohol_diagnosis',
    columns='LISTYR',
    aggfunc=len
)

In [88]:
df_alc_by_listyr_uniq

LISTYR,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
alcohol_diagnosis,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
False,1548,2016,2506,2924,3240,3623,3766,4330,4750,5163,5563,6315,6377,6513,5839,6416,6646,6776,6832,7149,7143,7441,7808,7652,7473,7804,7739,7616,7888,7856,7919,7728,7184,7256,7122,7580,4502
True,126,270,464,519,620,751,1215,1638,1877,1896,2267,2404,2421,2313,1958,2088,2344,2468,2550,2359,2364,2281,2508,2594,2646,2725,2902,3128,3605,3819,4072,4501,4799,5558,5561,6141,2682


In [89]:
df_alc_by_listyr = df_alc_by_listyr_uniq.T.reset_index()
df_alc_by_listyr.columns = ['LISTYR', 'alcohol_diag_no', 'alcohol_diag_yes']

df_alc_by_listyr

,LISTYR,alcohol_diag_no,alcohol_diag_yes
0,1988.0,1548,126
1,1989.0,2016,270
2,1990.0,2506,464
3,1991.0,2924,519
4,1992.0,3240,620
5,1993.0,3623,751
6,1994.0,3766,1215
7,1995.0,4330,1638
8,1996.0,4750,1877
9,1997.0,5163,1896


In [90]:
df_alc_by_listyr['alcohol_diag_pct'] = (df_alc_by_listyr['alcohol_diag_yes'] / (df_alc_by_listyr['alcohol_diag_yes'] + df_alc_by_listyr['alcohol_diag_no'])) * 100

In [91]:
df_alc_by_listyr

,LISTYR,alcohol_diag_no,alcohol_diag_yes,alcohol_diag_pct
0,1988.0,1548,126,7.526882
1,1989.0,2016,270,11.811024
2,1990.0,2506,464,15.622896
3,1991.0,2924,519,15.074063
4,1992.0,3240,620,16.062176
5,1993.0,3623,751,17.169639
6,1994.0,3766,1215,24.392692
7,1995.0,4330,1638,27.446381
8,1996.0,4750,1877,28.323525
9,1997.0,5163,1896,26.859329


In [92]:
df_alc_by_listyr.to_clipboard(index=False)

In [93]:
# spot-checking 2023 numbers from source
test_23 = df_uniq_cands[df_uniq_cands['LISTYR'] == 2023]
test_23_has_alc = test_23[test_23['alcohol_diagnosis']]
test_23_no_alc = test_23[~test_23['alcohol_diagnosis']]
print(len(test_23_has_alc))
print(len(test_23_no_alc))

6141
7580
